In [1]:
import pandas as pd
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig,TrainingArguments,Trainer
from torch.utils.data import DataLoader, Dataset, random_split
from tqdm import tqdm
conversations_df = pd.read_csv("formatted_csv.csv")

In [2]:
from sklearn.model_selection import train_test_split
# 随机划分训练集和剩余数据
train_data, remaining_data = train_test_split(conversations_df, train_size=0.20, random_state=42)

# 保存训练集为新的CSV文件
train_data.to_csv("train_data.csv", index=False)

In [10]:
train_data.head()

,Speaker1,Speaker2
10447,i chose the wrong passage .,she can see the beast !
12561,hey ! we re gettin married !,right now everything i own !
15863,what do you mean ?,did you masturbate ?
13936,she called you george george .,. . .did i win some contest ?
7939,tradition .,it s all we have .


In [3]:
class ConversationDataset(Dataset):
    def __init__(self, conversations, tokenizer, max_length):
        self.conversations = conversations
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.conversations)
    
    def __getitem__(self, idx):
        conversation = self.conversations.iloc[idx]
        user_input = str(conversation['Speaker1'])  # Convert to string
        machine_output = str(conversation['Speaker2'])  # Convert to string
        
        user_input_ids = self.tokenizer.encode(
            user_input,
            add_special_tokens=True,
            max_length=self.max_length,
            pad_to_max_length=True,
            truncation=True
        )
        
        machine_output_ids = self.tokenizer.encode(
            machine_output,
            add_special_tokens=True,
            max_length=self.max_length,
            pad_to_max_length=True,
            truncation=True
        )
        
        return {
            "user_input_ids": torch.tensor(user_input_ids, dtype=torch.long),
            "machine_output_ids": torch.tensor(machine_output_ids, dtype=torch.long)
        }


In [4]:
# 1. 加载预训练的BART模型和分词器
model_name = "facebook/bart-large"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [5]:
# 请确保你已经有一个合适的数据集对象，这里假设你的数据集对象是conversations_dataset
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

In [6]:
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=1e-4,
    save_total_limit=2,
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [11]:
trainer.train()

  0%|          | 0/3147 [02:52<?, ?it/s]


KeyError: 4145